In [3]:
import pystan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts

In [4]:
df = pd.read_csv("/Users/oscarengelbrektson/Downloads/CS146 LBA data gathering (Fall 2019) (Responses) - Form Responses 1 (1).csv",
                header=1)

In [5]:
df.iloc[1, 3]

'ALDI'

The desired outcome is a dataset with one row per observation. For each observation there should be a column representing the outcome variable, price, and the predictor variables, food, location, store, respectively.

To convert the above dataset to this desired format the following steps are taken:
1. We normalize each product price by dividing by quantity.
2. Currently each row contains up to 3 observations for each of the 10 foods. For every food, we iterate over the three observations reported by each student at each location
3. Take note of the store and the location at where the observation was recorded
4. Add it as a row in the new dataset
5. This procedure assumes that each student recorded 3 products/observations for every foods. This is not always the case, if there were not 3 observations students were asked to leave the price empty. Hence, to remove these non-observations, we delete all rows were the price is Nan/missing.

In [ ]:
column_names = ["Price", "Food", "Location", "Store"]
data = pd.DataFrame(columns=column_names)

quantity_col_indexes = list(range(5, len(df.columns), 2))
for row in range(len(df)):
    for i in quantity_col_indexes:
        prices.append(df.iloc[row,i+1]/df.iloc[row,i])
        foods.append(df.iloc[row])

col_to_food = {5:"Apples", 11:"Bananas", 17:"Tomatoes", 23:"Potatoes", 29:"Flour, white", 
         35:"Rice, basmati", 41:"Milk, full cream", 47:"Butter", 53:"Eggs", 59:"Chicken breasts"}
    

foods = ["Apples", "Bananas", "Tomatoes", "Potatoes", "Flour, white", 
         "Rice, basmati", "Milk, full cream", "Butter", "Eggs", "Chicken breasts"]

quantity_col_indexes = list(range(5, len(df.columns), 2))
for i in quantity_col_indexes:
    df.iloc[:,i+1] = df.iloc[:,i+1]/df.iloc[:,i]

#Drop quantity columns
df.drop(df.columns[quantity_col_indexes], axis=1, inplace=True)
food_prices = {}
for i in range(len(foods)):
    #get prices for each food product
    prices = df.iloc[:,5+3*i:5+3*i+3].values.flatten()
    
    #Remove nan entries and add to dictionary
    food_prices[foods[i]] = prices[~np.isnan(prices)]

In [14]:
prices, foods, locations, stores = [], [], [], []
col_to_food = {5:"Apples", 11:"Bananas", 17:"Tomatoes", 23:"Potatoes", 29:"Flour, white", 
         35:"Rice, basmati", 41:"Milk, full cream", 47:"Butter", 53:"Eggs", 59:"Chicken breasts"}

quantity_col_indexes = list(range(5, len(df.columns), 6))
for i in quantity_col_indexes:
    food = col_to_food[i]
    
    for row in range(len(df)):
        location = df.iloc[row, 4]
        store = df.iloc[row, 3]
        
        #3 products for every food
        for j in range(0, 6, 2):
            
            normalized_price = df.iloc[row, i+j]/df.iloc[row, i+j+1]
            
            #Record values
            if not np.isnan(normalized_price):
                prices.append(normalized_price)
                foods.append(food)
                locations.append(location)
                stores.append(store)

In [15]:
dataset = pd.DataFrame()
dataset["prices"] = prices
dataset["foods"] = foods
dataset["locations"] = locations
dataset["stores"] = stores

In [16]:
dataset.head()

,prices,foods,locations,stores
0,0.400000,Apples,Rummelsburger Str. 98,ALDI
1,0.531915,Apples,Rummelsburger Str. 98,ALDI
2,0.317460,Apples,Rummelsburger Str. 98,ALDI
3,0.401606,Apples,Karl-Marx-Straße 92-98,REWE
4,0.671141,Apples,Karl-Marx-Straße 92-98,REWE


In [ ]:
for row in range(len(df)):
    for i in quantity_col_indexes:
        prices.append(df.iloc[row,i+1]/df.iloc[row,i])
        foods.append(df.iloc[row])

In [ ]:
list(zip(prices, foods, locations, stores))

In [ ]:
np.array([[1, 2, 3, 4], [5, 6, 7, 8]]).T

In [ ]:
dataset = []

for item in goods:
    for _, row in data.iterrows():
        for i in range(1, 4):
            if not isnan(row["{:s}_price_{:d}".format(item, i)]):
                dataset.append([str(item),
                                  row["{:s}_quantity_{:d}".format(item, i)], 
                                  row["{:s}_price_{:d}".format(item, i)], 
                                  row['Grocery store'], 
                                  row['Grocery store street address']])

dataset = pd.DataFrame(dataset, columns=['Product','Quantity', 'Price', 'Store', 'Location'])
dataset.head(10)

In [ ]:
quantity_col_indexes = list(range(5, len(df.columns), 2))
for row in range(len(df)):
    prices.append(df.iloc[row,2]/df.iloc[row,1])

In [ ]:
#Place the 4 German stores first
stores = ['ALDI', 'REWE', 'Lidl', 'EDEKA','Tesco Express', 'Waitrose & Partners',
         'Safeway', 'Waitrose', 'Lotte Mart', 'Tesco', "SAINSBURY'S",
         'Waitrose & Partners ', "Sainsbury's ", 'Sainsbury', 'Sainsbury‚Äôs ',
         'Waitrose & Partners Islington', 'Tesco ']

unique_addresses = dataset.location.unique()
address_dict = {}
postcode_dict = {}

rep = re.compile('|'.join(map(re.escape, stores)))
for i in range(len(unique_addresses)):
    
    #If german store and Berlin not in adress, add Berlin at the end
    if dataset.iloc[dataset.loc[dataset.location==unique_addresses[i]].index[0], 3] in stores[:4] \
    and 'Berlin' not in unique_addresses[i]:
        unique_addresses[i] = unique_addresses[i] + " Berlin"
        
    #If present, remove store name
    unique_addresses[i] = rep.sub('', unique_addresses[i])
    
    #For some reason, the German double "s" gets converted to "√∂" or "√ü" which throws errors
    if "√∂" in unique_addresses[i]:
        unique_addresses[i] = unique_addresses[i].replace("√∂", "ö")
    if "√ü" in unique_addresses[i]:
        unique_addresses[i] = unique_addresses[i].replace("√ü", "ß")
        
    #The API doesn't recognize "Barbican" in London, so remove it from the location tag
    if "Barbican" in unique_addresses[i]:
        unique_addresses[i] = unique_addresses[i].replace("Barbican", "")
    
    
    #Get the full location information and add the relevant pieces to the appropriate dicts
    full_location = nom.geocode(unique_addresses[i], addressdetails=True)
    postcode_dict[unique_addresses[i]] = full_location.raw['address']['postcode']
    
    #Location data varies by region
    if full_location.raw['address']['country'] == 'Deutschland':
        try:
            address_dict[unique_addresses[i]] = full_location.raw['address']['city_district']
        except KeyError:
            address_dict[unique_addresses[i]] = full_location.raw['address']['town']
            
    elif full_location.raw['address']['country'] == 'United Kingdom':
        address_dict[unique_addresses[i]] = full_location.raw['address']['state_district']
        
    else:
        address_dict[unique_addresses[i]] = full_location.raw['address']['city']

In [ ]:
stan_code = """

data {
    int<lower=0> n;
    real<lower=0> price[n];
    
    int<lower=0> p;
    int<lower=1> product[n];
    
    int<lower=0> s;
    int<lower=1> store[n];
    
    int<lower=0> l;
    int<lower=1> location[n];
}

parameters {
    real<lower=0> sigma;
    
    real<lower=0> base_price[p];
    real<lower=0> s_multiplier[s];
    real<lower=0> l_multiplier[l];
}

model {

    base_price ~ cauchy(0, 10);
    sigma ~ normal(0, 1);
    
    s_multiplier ~ lognormal(0, .5);
    l_multiplier ~ lognormal(0, .5);
    
    for(i in 1:n) {
        price[i] ~ normal(base_price[product[i]] 
        * (s_multiplier[store[i]]) 
        * (l_multiplier[location[i]]), sigma);}
        
}
"""